In [72]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [283]:
# based on dataset from data cleaning
df = pd.read_csv('~/Desktop/303/cleaned_data.csv')
df.columns
df= df.fillna(df.mean())
#df = df.loc[df.rankingSortRank > 0,:]
df

,act_avg,city,sortName,zip,rankingDisplayScore,percent_receiving_aid,cost_after_aid,state,rankingSortRank,hs_gpa_avg,...,avg_cost,instante_tuition,outstate_tuition,tuition_revenue_per,instructional_expenditure_per,avg_faculty_salary,ft_faculty_rate,avg_net_price,number_titleIV,sat_avg
0,32.0,Princeton,princetonuniversity,8544,100.000000,60.00000,16793.000000,NJ,1,3.900000,...,64390.0,47140.0,47140.0,13216.0,53658.0,17519.0,0.8490,10027.0,346.0,1500.0
1,32.0,Cambridge,harvarduniversity,2138,98.000000,55.00000,16338.000000,MA,2,4.000000,...,66900.0,48949.0,48949.0,31930.0,43823.0,19741.0,0.8589,14327.0,350.0,1523.0
2,32.0,Chicago,universityofchicago,60637,96.000000,42.00000,27767.000000,IL,3,4.000000,...,72717.0,56034.0,56034.0,28508.0,89151.0,18115.0,0.8239,25455.0,494.0,1524.0
3,32.0,New Haven,yaleuniversity,6520,96.000000,50.00000,18385.000000,CT,3,3.546502,...,68950.0,51400.0,51400.0,17934.0,120956.0,17517.0,0.7459,18627.0,387.0,1520.0
4,32.0,New York,columbiauniversity,10027,95.000000,48.00000,21041.000000,NY,5,3.546502,...,71972.0,57208.0,57208.0,37294.0,89742.0,18713.0,0.4688,24231.0,526.0,1522.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,18.0,Carrollton,universityofwestgeorgia,30118,50.465217,35.27972,33920.867133,GA,-1,3.200000,...,21257.0,6226.0,17010.0,5755.0,7083.0,6452.0,0.8041,15056.0,1799.0,1016.0
296,20.0,Milwaukee,universityofwisconsinmilwaukee,53201,50.465217,35.27972,33920.867133,WI,-1,3.100000,...,21701.0,9565.0,20844.0,8765.0,8925.0,8426.0,0.6545,16650.0,1951.0,1125.0
297,19.0,Valdosta,valdostastateuniversity,31698,50.465217,35.27972,33920.867133,GA,-1,3.200000,...,20988.0,6410.0,17196.0,6216.0,6718.0,6624.0,0.7658,14656.0,941.0,1059.0
298,21.0,Wichita,wichitastateuniversity,67260,50.465217,35.27972,33920.867133,KS,-1,3.400000,...,18924.0,8432.0,17390.0,8568.0,7543.0,7651.0,0.9680,14113.0,777.0,1159.0


In [284]:
# selecting columns in int and float type for loocv
X = df[['act_avg', 'sat_avg','rankingDisplayScore', 'percent_receiving_aid',
       'cost_after_aid', 'hs_gpa_avg', 'xwalkId','businessRepScore', 'tuition',
       'engineeringRepScore','overallRank',
       'primaryKey', 'OPEID6','branches', 'admission_rate', 'satCR25', 'satCR75', 'satmt25', 'satmt75',
       'satcrmid', 'satmtmid', 'actcm25', 'actcm75', 'acten25', 'acten75',
       'actmt25', 'actmt75', 'actcmmid', 'actenmid', 'actmtmid',
       'ug_enrollment', 'percent_white', 'percent_black', 'percent_hispanic',
       'percent_asian', 'percent_aian', 'percent_nhpi', 'percent_twoormore',
       'percent_nra', 'percent_unknown', 'percent_parttime', 'avg_cost',
       'instante_tuition', 'outstate_tuition', 'tuition_revenue_per',
       'instructional_expenditure_per', 'avg_faculty_salary',
       'ft_faculty_rate', 'avg_net_price', 'number_titleIV', 'sat_avg']]
Y = df[['rankingSortRank']]

In [285]:
X.dtypes

act_avg                          float64
sat_avg                          float64
rankingDisplayScore              float64
percent_receiving_aid            float64
cost_after_aid                   float64
hs_gpa_avg                       float64
xwalkId                            int64
businessRepScore                 float64
tuition                            int64
engineeringRepScore              float64
overallRank                        int64
primaryKey                         int64
OPEID6                             int64
branches                           int64
admission_rate                   float64
satCR25                          float64
satCR75                          float64
satmt25                          float64
satmt75                          float64
satcrmid                         float64
satmtmid                         float64
actcm25                          float64
actcm75                          float64
acten25                          float64
acten75         

In [286]:
# use loocv to predict and get accuracy
X = df[['act_avg', 'sat_avg','rankingDisplayScore', 'percent_receiving_aid',
       'cost_after_aid', 'hs_gpa_avg', 'xwalkId','businessRepScore', 'tuition',
       'engineeringRepScore','overallRank',
       'primaryKey', 'OPEID6','branches', 'admission_rate', 'satCR25', 'satCR75', 'satmt25', 'satmt75',
       'satcrmid', 'satmtmid', 'actcm25', 'actcm75', 'acten25', 'acten75',
       'actmt25', 'actmt75', 'actcmmid', 'actenmid', 'actmtmid',
       'ug_enrollment', 'percent_white', 'percent_black', 'percent_hispanic',
       'percent_asian', 'percent_aian', 'percent_nhpi', 'percent_twoormore',
       'percent_nra', 'percent_unknown', 'percent_parttime', 'avg_cost',
       'instante_tuition', 'outstate_tuition', 'tuition_revenue_per',
       'instructional_expenditure_per', 'avg_faculty_salary',
       'ft_faculty_rate', 'avg_net_price', 'number_titleIV']]
y = df[['rankingSortRank']]
loocv = LeaveOneOut()
loocv.get_n_splits(X)

model = LogisticRegression()

true = []
predicted = []

for train_index, test_index in loocv.split(X):

    X_train=X.loc[train_index]
    X_test=X.loc[test_index]
    y_train=y.loc[train_index]
    y_test=y.loc[test_index]
    
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    true.append(y_test['rankingSortRank'].values[0])
    predicted.append(y_pred[0])
    
print(true)
print("-----")
print(predicted)
accuracy = accuracy_score(true, predicted)

print("Accuracy:", accuracy) # accuracy score for specific rank predicting, will do classification accuracy later

[1, 2, 3, 3, 5, 5, 5, 8, 9, 10, 11, 11, 11, 14, 14, 14, 14, 18, 18, 20, 21, 21, 21, 21, 25, 25, 27, 28, 29, 30, 30, 32, 32, 34, 34, 34, 37, 37, 37, 40, 40, 42, 42, 42, 42, 46, 46, 46, 46, 46, 46, 52, 52, 54, 54, 56, 56, 56, 56, 56, 61, 61, 61, 61, 61, 61, 67, 68, 69, 69, 69, 69, 69, 69, 75, 75, 75, 78, 78, 78, 81, 81, 81, 81, 81, 81, 87, 87, 87, 90, 90, 90, 90, 94, 94, 94, 97, 97, 97, 97, 97, 97, 103, 103, 103, 103, 103, 103, 103, 110, 110, 110, 110, 110, 115, 115, 115, 115, 115, 120, 120, 120, 120, 124, 124, 124, 124, 124, 124, 124, 124, 132, 133, 133, 133, 133, 133, 133, 133, 140, 140, 140, 140, 140, 145, 145, 145, 145, 145, 145, 151, 151, 151, 151, 151, 156, 156, 156, 159, 159, 159, 159, 159, 159, 165, 165, 165, 165, 165, 165, 171, 171, 171, 171, 171, 176, 176, 176, 176, 176, 181, 181, 181, 181, 181, 181, 187, 187, 187, 187, 187, 192, 192, 192, 192, 192, 192, 198, 198, 198, 198, 202, 202, 202, 202, 202, 207, 207, 207, 207, 207, 207, 207, 207, 207, 216, 216, 216, 216, 216, 216, 216, 

In [287]:
# use loocv to predict and get accuracy
X = df[['act_avg', 'sat_avg','rankingDisplayScore', 'percent_receiving_aid',
       'cost_after_aid', 'hs_gpa_avg', 'xwalkId','businessRepScore', 'tuition',
       'engineeringRepScore','overallRank',
       'primaryKey', 'OPEID6','branches', 'admission_rate', 'satCR25', 'satCR75', 'satmt25', 'satmt75',
       'satcrmid', 'satmtmid', 'actcm25', 'actcm75', 'acten25', 'acten75',
       'actmt25', 'actmt75', 'actcmmid', 'actenmid', 'actmtmid',
       'ug_enrollment', 'percent_white', 'percent_black', 'percent_hispanic',
       'percent_asian', 'percent_aian', 'percent_nhpi', 'percent_twoormore',
       'percent_nra', 'percent_unknown', 'percent_parttime', 'avg_cost',
       'instante_tuition', 'outstate_tuition', 'tuition_revenue_per',
       'instructional_expenditure_per', 'avg_faculty_salary',
       'ft_faculty_rate', 'avg_net_price', 'number_titleIV']]
y = df[['rankingSortRank']]
loocv = LeaveOneOut()
loocv.get_n_splits(X)

model = LogisticRegression()

true = []
predicted = []

high_df = []
low_df = []
high_predicted = []
low_predicted = []

for train_index, test_index in loocv.split(X):

    X_train=X.loc[train_index]
    X_test=X.loc[test_index]
    y_train=y.loc[train_index]
    y_test=y.loc[test_index]
    
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    true.append(y_test['rankingSortRank'].values[0])
    predicted.append(y_pred[0])
    
    #classifying into high and low
    
    if y_test['rankingSortRank'].values[0] > 50:
        high_df.append(test_index[0])
    else:
        low_df.append(test_index[0])
    if y_pred[0] > 50:
        high_predicted.append(test_index[0])
    else:
        low_predicted.append(test_index[0])

print(high_df)
print("-----")
print(low_df)
print("prediction below-----------------")
print(high_predicted)
print("-----")
print(low_predicted)

[51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229]
-----
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 

In [288]:
correct_h = 0
for i in high_df:
    for j in high_predicted:
        if i ==j:
            correct_h = correct_h + 1 
correct_h            

136

In [289]:
correct_l = 0
for i in low_df:
    for j in low_predicted:
        if i ==j:
            correct_l = correct_l + 1 
correct_l   

96

In [290]:
# accuracy rate for classifying
(correct_h +correct_l)/df.shape[0]

0.7733333333333333

In [296]:
# dataset for submodel
predicted_h = df.iloc[[27, 29, 32, 34, 37, 38, 39, 41, 44, 49, 50, 51, 52, 53, 54, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 89, 90, 91, 93, 94, 95, 96, 97, 98, 99, 100, 102, 103, 104, 105, 106, 107, 108, 109, 110, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 156, 157, 158, 159, 160, 162, 163, 166, 167, 168, 169, 171, 172, 173, 174, 176, 179, 181, 182, 183, 184, 185, 186, 189, 191, 192, 193, 195, 196, 198, 199, 203, 204, 205, 210, 211, 212, 214, 215, 218, 219, 220, 227, 229, 231, 232, 235, 241, 247, 253, 254, 262, 272, 277, 284, 286, 290, 293],:]
predicted_h

,act_avg,city,sortName,zip,rankingDisplayScore,percent_receiving_aid,cost_after_aid,state,rankingSortRank,hs_gpa_avg,...,avg_cost,instante_tuition,outstate_tuition,tuition_revenue_per,instructional_expenditure_per,avg_faculty_salary,ft_faculty_rate,avg_net_price,number_titleIV,sat_avg
27,29.0,Ann Arbor,universityofmichiganannarbor,48109,74.000000,25.00000,32856.000000,MI,28,3.800000,...,28652.0,14826.0,47476.0,23581.0,25620.0,11258.0,0.8275,14860.0,1606.0,1436.0
29,29.0,New York,newyorkuniversity,10012,71.000000,46.00000,41783.000000,NY,30,3.600000,...,67865.0,50464.0,50464.0,35390.0,30614.0,13719.0,0.9990,40336.0,2763.0,1408.0
32,28.0,Williamsburg,collegeofwilliamandmary,23187,70.000000,21.00000,42153.000000,VA,32,3.546502,...,35991.0,22044.0,43670.0,20919.0,15105.0,11189.0,0.7987,19720.0,368.0,1406.0
34,30.0,Atlanta,georgiainstituteoftechnology,30332,68.000000,15.00000,39518.000000,GA,34,3.900000,...,27245.0,12418.0,33014.0,13590.0,12566.0,12691.0,0.8908,13291.0,560.0,1389.0
37,30.0,Cleveland,casewesternreserveuniversity,44106,67.000000,48.00000,35248.000000,OH,37,3.546502,...,63216.0,47500.0,47500.0,25887.0,29979.0,11400.0,0.7824,35316.0,806.0,1441.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,17.0,Princess Anne,universityofmarylandeasternshore,21853,50.465217,35.27972,33920.867133,MD,-1,2.900000,...,22192.0,8042.0,18048.0,4214.0,12436.0,8235.0,0.6935,15860.0,479.0,950.0
284,19.0,Greeley,universityofnortherncolorado,80639,50.465217,35.27972,33920.867133,CO,-1,3.300000,...,22604.0,9546.0,21132.0,10560.0,10145.0,7749.0,0.6081,20367.0,1233.0,1107.0
286,20.0,Mobile,universityofsouthalabama,36688,50.465217,35.27972,33920.867133,AL,-1,3.500000,...,19702.0,7812.0,15324.0,9198.0,9082.0,7203.0,0.6234,15082.0,1158.0,1142.0
290,18.0,Edinburg,universityoftexasriograndevalley,78539,50.465217,35.27972,33920.867133,TX,-1,3.546502,...,13401.0,7587.0,17547.0,4447.0,7266.0,7573.0,0.8718,3901.0,2820.0,1020.0


In [298]:
# dataset for submodel
predicted_l = df.iloc[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 30, 31, 33, 35, 36, 40, 42, 43, 45, 46, 47, 48, 55, 57, 68, 87, 88, 92, 101, 111, 112, 132, 140, 141, 155, 161, 164, 165, 170, 175, 177, 178, 180, 187, 188, 190, 194, 197, 200, 201, 202, 206, 207, 208, 209, 213, 216, 217, 221, 222, 223, 224, 225, 226, 228, 230, 233, 234, 236, 237, 238, 239, 240, 242, 243, 244, 245, 246, 248, 249, 250, 251, 252, 255, 256, 257, 258, 259, 260, 261, 263, 264, 265, 266, 267, 268, 269, 270, 271, 273, 274, 275, 276, 278, 279, 280, 281, 282, 283, 285, 287, 288, 289, 291, 292, 294, 295, 296, 297, 298, 299],:]
predicted_l

,act_avg,city,sortName,zip,rankingDisplayScore,percent_receiving_aid,cost_after_aid,state,rankingSortRank,hs_gpa_avg,...,avg_cost,instante_tuition,outstate_tuition,tuition_revenue_per,instructional_expenditure_per,avg_faculty_salary,ft_faculty_rate,avg_net_price,number_titleIV,sat_avg
0,32.0,Princeton,princetonuniversity,8544,100.000000,60.00000,16793.000000,NJ,1,3.900000,...,64390.0,47140.0,47140.0,13216.0,53658.0,17519.0,0.8490,10027.0,346.0,1500.0
1,32.0,Cambridge,harvarduniversity,2138,98.000000,55.00000,16338.000000,MA,2,4.000000,...,66900.0,48949.0,48949.0,31930.0,43823.0,19741.0,0.8589,14327.0,350.0,1523.0
2,32.0,Chicago,universityofchicago,60637,96.000000,42.00000,27767.000000,IL,3,4.000000,...,72717.0,56034.0,56034.0,28508.0,89151.0,18115.0,0.8239,25455.0,494.0,1524.0
3,32.0,New Haven,yaleuniversity,6520,96.000000,50.00000,18385.000000,CT,3,3.546502,...,68950.0,51400.0,51400.0,17934.0,120956.0,17517.0,0.7459,18627.0,387.0,1520.0
4,32.0,New York,columbiauniversity,10027,95.000000,48.00000,21041.000000,NY,5,3.546502,...,71972.0,57208.0,57208.0,37294.0,89742.0,18713.0,0.4688,24231.0,526.0,1522.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,18.0,Carrollton,universityofwestgeorgia,30118,50.465217,35.27972,33920.867133,GA,-1,3.200000,...,21257.0,6226.0,17010.0,5755.0,7083.0,6452.0,0.8041,15056.0,1799.0,1016.0
296,20.0,Milwaukee,universityofwisconsinmilwaukee,53201,50.465217,35.27972,33920.867133,WI,-1,3.100000,...,21701.0,9565.0,20844.0,8765.0,8925.0,8426.0,0.6545,16650.0,1951.0,1125.0
297,19.0,Valdosta,valdostastateuniversity,31698,50.465217,35.27972,33920.867133,GA,-1,3.200000,...,20988.0,6410.0,17196.0,6216.0,6718.0,6624.0,0.7658,14656.0,941.0,1059.0
298,21.0,Wichita,wichitastateuniversity,67260,50.465217,35.27972,33920.867133,KS,-1,3.400000,...,18924.0,8432.0,17390.0,8568.0,7543.0,7651.0,0.9680,14113.0,777.0,1159.0
